In [33]:
  
import numpy as np


#Added logistic
#Add regressor
class NeuralNetwork:
    def __init__(self, layer_sizes, layer_activations, epsilon = 0.1, lr=0.01):
        self.epsilon = epsilon
        self.layer_sizes = layer_sizes
        self.layer_activations = layer_activations
        self.initialize()
        self.lr = lr
        self.activation_functions = {
            "relu":self.relu,
            "sigmoid":self.sigmoid,
            "grelu":self.grelu,
            "gsigmoid":self.gsigmoid            
            
            }
        
    def initialize(self):
        self.W = [] #NN weights
        self.b = [] #NN betas
        for i in range(len(self.layer_sizes)-1):
            self.W.append(np.random.randn(self.layer_sizes[i+1],self.layer_sizes[i]  )*self.epsilon) #Randomly initializes weights for all layers 
            self.b.append(np.random.randn(self.layer_sizes[i+1],1)*self.epsilon) #Randomly initializes betas for all layers 
            #print(self.W[i])
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    
    
    def gsigmoid(self,da,x):
        temp = self.sigmoid(x)
   
        return da * temp * (1-temp)
    
    
    def relu(self,x):
        return np.maximum(0,x)
    def grelu(self,da,x):

        temp = np.array(da, copy=True)
        temp[x<=0] = 0
        return temp
    

    def propagate(self,x,is_pred=False):
        cacheW = []
     
        z = x.T
       
      
        #print(z)
        a = self.activation_functions[self.layer_activations[0]](z)
        
        
        for i in range(len(self.W)):
            cacheW.append((a,z))
            #print(self.W[i].shape)
        
           

           
            z = np.dot(self.W[i], a ) + self.b[i]
            #print(self.b[i].shape)
           
            a = self.activation_functions[self.layer_activations[i]](z)
            #print(a.shape)
            assert(a.shape == z.shape)
        cacheW.append((a,z))

        if not is_pred:       
            
            self.cache =  cacheW
        else:
            return cacheW[-1][0]
  
    
    
    
    
    def backpropagate(self,y,y_pred):
        gradsW = []
        gradsb= []
     
        
        #print(y.shape, y_pred.shape)
        da= -(np.divide(y,y_pred+self.epsilon) - np.divide(1-y,1-y_pred+self.epsilon))
        #print(da.shape)
        
       
        for i in range(len(self.W))[::-1]:
            
            
          
            
            a_prev =  self.cache[i][0]
            #print(a.shape)
            
            #print(self.cache[0][0].shape)
           
          
           
            z = self.cache[i+1][1] #Need to get the z for the next layer
            #print(z.shape)
            #print(a.shape)
            
            w = self.W[i]
          
            b = self.b[i]
            
            dz = self.activation_functions["g"+self.layer_activations[i]](da,z)
          
            #print(dz.shape)
            #print(a.shape)
            
            
         
            
            dw = np.dot(dz, a_prev.T)/a_prev.shape[1]
            
            db = np.sum(dz, axis=1,keepdims=True)/a_prev.shape[1]
           
        
            #print(dw.shape)
           
            da = np.dot(w.T, dz)
            
            #print(a.shape)
           
            #print(dw)
            
            gradsW.append(dw)
            gradsb.append(db)
            
        self.gradsw = gradsW
        self.gradsb = gradsb
        
    def update(self):
        
       
        
       
        for i in range(len(self.gradsw)):
            #print(self.gradsw[i])
            #print(self.gradsw[i].shape)
         
           
            self.W[i] -= self.lr*self.gradsw[-(i+1)] #It updates the weights of the first layer, second layer... the grads lists are inverted because of the way they were stored (last first)
            self.b[i] -= self.lr*self.gradsb[-(i+1)]
        #print(self.W[-1])
          
            
    def fit(self,x,y,epochs=1):
        for i in range(epochs):
            self.propagate(x)
            self.backpropagate(y,self.cache[-1][0])
            self.update()
            
    def predict(self,x):
        a =self.propagate(x,is_pred=True)
        return a


In [34]:
from sklearn.model_selection import train_test_split

In [38]:
import pandas as pd

df = pd.read_csv("C:\\Users\\Weriko\\source\\repos\\MachineLearning\MachineLearning\\train_mod2.csv")


In [58]:
%%time

X = df[["Pclass","Sex","Age","Fare"]].to_numpy()
y = df["Survived"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.15, random_state=42)
NN = NeuralNetwork((4,6,4,1),("relu","relu","sigmoid"), lr =0.01)
#print(NN.W)
NN.fit(X_train,y_train,epochs = 10000)


ypreds = np.array([0 if i <0.5 else 1 for i in NN.predict(X_test)[0]])
#print(NN.predict(X_test))
print(sum(ypreds==y_test)/y_test.shape[0])
#print(NN.W) 
#print(NN.b)


0.7985074626865671
Wall time: 2.1 s


In [48]:
from sklearn.neural_network import MLPClassifier


In [57]:
%%time
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, max_iter=10000,
                     hidden_layer_sizes=(6, 4))
clf.fit(X_train,y_train)
skypreds = clf.predict(X_test)
print(sum(skypreds==y_test)/y_test.shape[0])

0.7313432835820896
Wall time: 157 ms
